<a href="https://colab.research.google.com/github/beginner003/developing-with-github/blob/master/Naive_Bayes_Class_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if __name__ == '__main__':
  from google.colab import drive
  drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/COMP 2211/Lab2"

/content/drive/MyDrive/COMP 2211/Lab2


In [ ]:
%ls

lab2_review.ipynb  Lab2work.ipynb     test_labels.csv     train_labels.csv
lab2_task.ipynb    test_features.csv  train_features.csv


In [ ]:
import numpy as np


In [ ]:
if __name__ == '__main__':
  X_train = np.loadtxt("train_features.csv", delimiter=",", skiprows=1)
  X_test = np.loadtxt("test_features.csv", delimiter=",", skiprows=1)
  y_train = np.loadtxt("train_labels.csv", delimiter=",", skiprows=1)
  y_test = np.loadtxt("test_labels.csv", delimiter=",", skiprows=1)

In [ ]:
# Task 1: Estimate priors

def estimate_priors(y_train):
  # y_train: a 1D numpy array with shape (num_train_samples, )

  # TODO start
  trainsize= len(y_train)
  classes= np.unique(y_train)
  class_priors = np.empty((len(classes),))
  classindex=0
  for i in classes:
    count=0
    for j in y_train:
      if i==j:count+=1

    class_priors[classindex]=count/trainsize
    classindex+=1

  # TODO end

  return class_priors
  # class priors: a 1D numpy array with shape (num_classes, )

In [ ]:
# trainsize= len(y_train)
# classes= np.unique(y_train)
# class_priors=np.empty((len(classes),))
# classindex=0
# for i in classes:
#   count=0
#   for j in y_train:
#     if i==j:
#       count+=1
#   class_priors[classindex]=count/trainsize
#   classindex+=1

# print(class_priors)



[0.88446963 0.11553037]


In [ ]:
# Task 2: Likelihood for categorical feature

def cat_likelihood(X_train_feature, y_train, X_test_feature):
  # X_train_feature: a 1D numpy array with shape (num_train_samples, )
  # y_train: a 1D numpy array with shape (num_train_samples, )
  # X_test_feature: a 1D numpy array with shape (num_test_samples, )

  # TODO start
  feat_class = np.unique(X_train_feature)
  feat_class = np.sort(feat_class)

  belief= np.unique(y_train)
  belief=np.sort(belief)
  feat_likelihood = np.empty((len(X_test_feature),len(belief)))

  likelihoodarr=np.empty((len(feat_class),len(belief)))
  for i in range(0,len(feat_class)):
    for j in range(0,len(belief)):
      likelihoodarr[i,j]= (np.dot(X_train_feature == feat_class[i], (y_train == belief[j] ).astype(int))+1)/(np.sum(y_train == belief[j])+len(feat_class))

  for i in range(0,len(X_test_feature)):
      feat_likelihood[i]= likelihoodarr[int(X_test_feature[i])]

  # TODO end

  return feat_likelihood
  # feat_likelihood: a 2D numpy array with shape (num_test_samples, num_classes)

In [ ]:

# feat_class = np.unique(X_train[:, 0])
# np.sort(feat_class)
# print(feat_class)

# belief= np.unique(y_train)
# likelihoodarr=np.empty((len(feat_class),len(belief)))

# for i in range(0,len(feat_class)):
#   for j in range(0,len(belief)):
#     likelihoodarr[i,j]= (np.dot(X_train[:,0] == feat_class[i], (y_train == belief[j] ).astype(int))+1)/(np.sum(y_train == belief[j])+len(feat_class))

# X_test_feat = X_test[:,0]
# feat_likelihood = np.empty((len(X_test_feat),len(belief)))
# for i in range(0,13007):

#   feat_likelihood[i]= likelihoodarr[int(X_test_feat[i])]

# print(likelihoodarr)
# print(feat_likelihood)


[0. 1. 2.]
[[0.2737297  0.35423197]
 [0.61026673 0.5292106 ]
 [0.11600358 0.11655742]]
[[0.2737297  0.35423197]
 [0.61026673 0.5292106 ]
 [0.11600358 0.11655742]
 ...
 [0.2737297  0.35423197]
 [0.61026673 0.5292106 ]
 [0.61026673 0.5292106 ]]


In [ ]:
#Task 3:
def num_likelihood(X_train_feature, y_train, X_test_feature):
  # X_train_feature: a 1D numpy array with shape (num_train_samples, )
  # y_train: a 1D numpy array with shape (num_train_samples, )
  # X_test_feature: a 1D numpy array with shape (num_test_samples, )
  belief = np.unique(y_train)
  belief = np.sort(belief)
  def feature_mean(X_train_feature, y_train):
    # TODO Part 1 start

    mean = np.empty((len(belief),))
    for i in range(0,len(belief)):
      mean[i]=np.dot(X_train_feature,(y_train == belief[i]).astype(int))/np.sum(y_train==belief[i])
    # TODO Part 1 end
    return mean
    # mean: a 1D numpy array with shape (num_classes, )

  def feature_var(X_train_feature, y_train):
    # TODO Part 2 start
    var = np.empty((len(belief)))
    for i in range(0,len(belief)):
      mask = X_train_feature[(y_train==belief[i])]

      mask = [(x-mean[i])**2 for x in mask]

      var[i]=np.sum(mask)/len(mask)
    # TODO Part 2 end
    return var
    # var: a 1D numpy array with shape (num_classes, )

  mean = feature_mean(X_train_feature, y_train)

  var = feature_var(X_train_feature, y_train)

  # TODO Part 3 start
  feat_likelihood=np.empty((len(X_test_feature),len(belief)))
  for i in range(0,len(belief)):
    temp = [(-((x-mean[i])**2)/(2*var[i]))for x in X_test_feature]
    temp= np.exp(temp)
    temp = [(x/np.sqrt(2*np.pi*var[i]))for x in temp]
    feat_likelihood[:,i]=np.transpose(temp)
  # TODO Part 3 end

  return feat_likelihood
  # feat_likelihood: a 2D numpy array with shape (num_test_samples, num_classes)

In [ ]:
# x=np.array([53.8,43.2,67.9,67.2,90.3,20.2])
# y=np.array([1,0,1,0,1,1])
# xtest=np.array([90.5,78.2,44.4,33.3,60.2])
# num_likelihood(x,y,xtest)

array([[0.0004392 , 0.00695081],
       [0.00529681, 0.01146028],
       [0.02217377, 0.01358119],
       [0.00628771, 0.00976944],
       [0.03048103, 0.0156288 ]])

In [ ]:

#Task 4:
def predict(X_train, y_train, X_test, feat_type_list):
  # X_train: a 2D numpy array with shape (num_train_samples, num_features)
  # y_train: a 1D numpy array with shape (num_train_samples, )
  # X_test: a 2D numpy array with shape (num_test_samples, num_features)
  # feat_type_list: a list of feature types in string form in the order of features in X_train and X_test
    # For example: feat_type_list = [ 'cat', 'cat', 'cat', 'cat', 'num', 'num', 'num']
  belief = np.unique(y_train)
  belief = np.sort(belief)
  sum=np.zeros((len(X_test),len(belief)))
  beliefprob = estimate_priors(y_train)
  print(beliefprob)
  for i in range(0,len(belief)):
    sum[:,i]=beliefprob[i]

  sum=np.log(sum)
  # print(sum)

  # TODO start
  for i in range(len(feat_type_list)):
    if feat_type_list[i]=='cat':
      # print("Category:")
      sum+=np.log(cat_likelihood(X_train[:,i],y_train,X_test[:,i]))
    elif feat_type_list[i]=='num':
      # print("Numerical")
      sum+=np.log(num_likelihood(X_train[:,i],y_train,X_test[:,i]))
    # print(sum)

  test_predict=[np.argmax(x)for x in sum]
  # print(test_predict)
  # TODO end

  return test_predict
  # test_predict: a 1D numpy array with shape (num_test_samples, )

In [ ]:
# x=np.array([[0.2,1.1],
#    [2.2,3.4],
#    [4.4,5.2]])

# y=np.array([2.2,3.3])

# for i in range(0,2):
#   x[:,i]=y[i]

# print(x)

# x=np.array(
#     [[5,6],
#    [4,3],
#    [3,2]])

# sum=[np.max(i)for i in x]
# print(sum)



[[2.2 3.3]
 [2.2 3.3]
 [2.2 3.3]]
[6, 4, 3]


In [ ]:
# if __name__ == '__main__':
#   # Task 1
#   priors = estimate_priors(y_train)
#   print('The priors are ', priors) # [0.88446963 0.11553037]

#   # Task 2
#   feat_likelihood = cat_likelihood(X_train[:, 0], y_train, X_test[:, 0])
#   print('The likelihood values for "marital status" for the first 5 test samples are', feat_likelihood[:5, :])
#   '''[[0.2737297  0.35423197]
#       [0.61026673 0.5292106 ]
#       [0.11600358 0.11655742]
#       [0.61026673 0.5292106 ]
#       [0.11600358 0.11655742]]'''

#   # Task 3
#   feat_likelihood = num_likelihood(X_train[:, 4], y_train, X_test[:, 4])
#   print('The likelihood values for "age" for the first 5 test samples are', feat_likelihood[:5, :])
#   '''[[0.00032595 0.00217531]
#       [0.01257581 0.01682234]
#       [0.03691429 0.02835094]
#       [0.03851424 0.03006992]
#       [0.01664062 0.0195927 ]]'''

#   # Task 4
#   y_pred = predict(X_train, y_train, X_test, feat_type_list=[ 'cat', 'cat', 'cat', 'cat', 'num', 'num', 'num'])
#   accuracy_score = np.sum(y_pred == y_test) / y_test.shape[0]
#   print('The ratio of correct predictions to the whole dataset is ', accuracy_score)
#   # 0.8794495271776736


The priors are  [0.88446963 0.11553037]
The likelihood values for "marital status" for the first 5 test samples are [[0.2737297  0.35423197]
 [0.61026673 0.5292106 ]
 [0.11600358 0.11655742]
 [0.61026673 0.5292106 ]
 [0.11600358 0.11655742]]
The likelihood values for "age" for the first 5 test samples are [[0.00032595 0.00217531]
 [0.01257581 0.01682234]
 [0.03691429 0.02835094]
 [0.03851424 0.03006992]
 [0.01664062 0.0195927 ]]
[0.88446963 0.11553037]
The ratio of correct predictions to the whole dataset is  0.8794495271776736
